# Calling the FOLIO API within Jupyter and FolioClient


### Requirements

1. Create `.env` or `.env.local` file
   ```
   TENANT_ID="TENANT-ID-GOES-HERE"
   OKAPI_URL="https://okapi-DOMAINGOESHERE.folio.ebsco.com"
   USERNAME="USERNAME-GOES-HERE"
   PW="PASSWORD-GOES-HERE"
   ```
2. Save file


### 🚨 IMPORTANT 🚨

- `.env` and `.env.local` are listed in `.gitignore`. Even so, be **VERY** careful to **NOT** commit your username and password to GitHub!
- If you change the env variables after running the notebook, you need to RESTART the notebook to load the new env variables.


In [ ]:
import os
import dotenv
import json
import folioclient
import random
import folio_uuid
from folio_uuid import FOLIONamespaces

dotenv.load_dotenv("../.env.local")
env = os.environ


In [ ]:
help(folioclient.FolioClient)

In [ ]:
# Create instance of FolioClient
fc = folioclient.FolioClient(
    tenant_id=env["TENANT_ID"],
    gateway_url=env["OKAPI_URL"],
    username=env["USERNAME"],
    password=env["PW"],
)

### GET

In [ ]:
help(fc.folio_get)

In [ ]:
# Example call with folio_get
one_instance = fc.folio_get(
    path="/instance-storage/instances",
    key="instances",
    query_params={"limit": 1},
)

print(one_instance)

In [ ]:
help(fc.folio_get_all)

In [ ]:
# Example GET call with folio_get_all. This handles offset for you. 
# folio_get_all creates a generator object that can only be iterated on once.
# folio_get_all requires patience for really big datasets!

all_users = fc.folio_get_all(
    path="/users",
    key="users",
)

all_user_ids = [i["id"] for i in all_users]

random.choice(all_user_ids)

In [ ]:
client = fc.get_folio_http_client()
help(client.get)

In [ ]:
# Example GET call with http client
resp = client.get(
    url=fc.gateway_url + "/instance-storage/instances",
    headers=fc.okapi_headers,
    params={"limit": 1},
)
one_instance = resp.json()["instances"]
print(one_instance)

In [ ]:
# Response handling with the http_client

client = fc.get_folio_http_client()
test_endpoints = [
    "/not-a-real-endpoint",
    "/instance-storage/instances"
]
for endpoint in test_endpoints:
    resp = client.get(
        url= fc.gateway_url + endpoint,
        headers=fc.okapi_headers,
        params={"limit":1}
    )
    print_statement = [str(resp.status_code), endpoint]
    if resp.status_code > 204:
        # If the call was unsuccessful...
        print_statement.append(resp.text)
    else:
        one_instance = resp.json()["instances"]
        print_statement.append(json.dumps(one_instance))
    print("\t".join(print_statement))

### POST, PUT, DELETE

In [ ]:
## BUILD FAKE USER TO UPLOAD, UPDATE, AND DELETE

fake_user_identifier = "test6789"

patron_group = fc.folio_get("/groups?query=group==zEBSCO*", "usergroups")[0]["id"]

user_id = str(
    folio_uuid.FolioUUID(
        okapi_url=fc.gateway_url,
        folio_object_type=FOLIONamespaces.users,
        legacy_identifier=fake_user_identifier,
    )
)

new_fake_user = {
    "id": user_id,
    "externalSystemId": fake_user_identifier,
    "username": fake_user_identifier,
    "barcode": fake_user_identifier,
    "patronGroup": patron_group,
    "personal": {"lastName": "Test", "firstName": "Data IC"},
}

In [ ]:
help(fc.folio_post)

In [ ]:
# Example POST call with folio_post
# You do NOT need to stringify the JSON for folio_post and folio_put

new_user = fc.folio_post(
    path="/users",
    payload=new_fake_user
)

In [ ]:
help(client.put)

In [ ]:
# Example PUT call with client
# You DO need to stringify the JSON for client.post or client.put

new_fake_user["barcode"] = "new_barcode_12345"

client = fc.get_folio_http_client()

resp = client.put(
    fc.gateway_url + f"/users/{new_fake_user['id']}",
    headers=fc.okapi_headers,
    data=json.dumps(new_fake_user),
)

print(resp.status_code)

In [ ]:
help(client.delete)

In [ ]:
# Example DELETE call with client.
# There is no folio_delete function (as of 08/29/2025)

client = fc.get_folio_http_client()
resp = client.delete(
    fc.gateway_url + f"/users/{new_fake_user['id']}",
    headers=fc.okapi_headers,
)

print(resp.status_code)